In [5]:
import pandas as pd
import os
from random import randint
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:

import shutil
from sklearn.model_selection import train_test_split

# Creating train,validation and test sets
data = pd.read_csv('data_labels.csv')
for _, row in data.iterrows():
    print(f'label: {row.labels}')
    print(f'Image name: {row.img_name}')
    break

dataset_dir = 'data/'
train_dir = 'data/train/'
val_dir = 'data/validation/'
test_dir = 'data/test/'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Split the DataFrame into training, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

train_split = 0.7
val_split = 0.15
test_split = 0.15

df = data.sample(frac=1, random_state=42)

total_images = len(df)
num_train = int(total_images * train_split)
num_val = int(total_images * val_split)
num_test = total_images - num_train - num_val

# Copy/move training images
for _, row in df[:num_train].iterrows():
    src = os.path.join(dataset_dir, row['img_name'])
    dst = os.path.join(train_dir, row['img_name'])
    shutil.copyfile(src, dst)

# Copy/move validation images
for _, row in df[num_train:num_train+num_val].iterrows():
    src = os.path.join(dataset_dir, row['img_name'])
    dst = os.path.join(val_dir, row['img_name'])
    shutil.copyfile(src, dst)

# Copy/move test images
for _, row in df[num_train+num_val:].iterrows():
    src = os.path.join(dataset_dir, row['img_name'])
    dst = os.path.join(test_dir, row['img_name'])
    shutil.copyfile(src, dst)

#Saving the df into csv files

train_df.to_csv('train_set.csv')
test_df.to_csv('test_set.csv')
val_df.to_csv('val_set.csv')

#########################

In [ ]:

data = pd.read_csv('data_labels.csv')
dir = 'data/'
#Visualizar a imagem antes do pre-processamento
image_files = [f for f in os.listdir(dir) if f.endswith(".jpg") or f.endswith(".png")]
count = 0
while count < 5:
    count += 1
    index = randint(0,len(data)-1)
    image_path = os.path.join(dir, data.loc[index,'img_name'])
    img = np.asarray(Image.open(image_path))
    plt.title(label = data.loc[index,'labels'])
    plt.imshow(img)
    plt.show()

In [ ]:
import torch
import torch.nn as nn

# definindo o modelo
test_1 = [224,128,64,32,16]
test_2 = [128,64,32,16,8]
test_3 = [64,32,16,8,4]
test_4 = [16,32,64,128,224]
test_5 = [64,32,16,32,8]

tests = [test_1,test_2,test_3,test_4,test_5]


# Defina uma semente para a inicialização
seed = 42
torch.manual_seed(seed)

import torch
import torch.nn as nn

# Defina uma semente para a inicialização
seed = 42
torch.manual_seed(seed)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class NeuralNetwork(nn.Module):
    def __init__(self, test, dropout_prob=0.5):
        super(NeuralNetwork, self).__init__()        
        # Tamanho da imagem de entrada
        self.input_size = (3, 224, 224)
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, test[0], kernel_size=3, padding=0),
            nn.BatchNorm2d(test[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(test[0], test[1], kernel_size=3, padding=0),
            nn.BatchNorm2d(test[1]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(test[1], test[2], kernel_size=3, padding=0),
            nn.BatchNorm2d(test[2]),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(test[2], test[3], kernel_size=3, padding=0),
            nn.BatchNorm2d(test[3]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(test[3], test[4], kernel_size=3, padding=0),
            nn.BatchNorm2d(test[4]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1)
        )

        self.dropout = nn.Dropout(p=dropout_prob)

        # Calcule o tamanho da camada linear de acordo com o tamanho de entrada
        self.flatten = Flatten()
        flattened_size = self._get_flattened_size(self.input_size)

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flattened_size, int(flattened_size/2)),
            nn.ReLU(),
            nn.Linear(int(flattened_size/2), 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

    def _get_flattened_size(self, input_size):
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_size)
            output = self.conv_layers(dummy_input)
            flattened_output = self.flatten(output)
            return flattened_output.size(1)


for i, test in enumerate(tests):
    model = NeuralNetwork(test)
    with torch.no_grad():
        input_size = (3, 224, 224)
        dummy_input = torch.zeros(1, *input_size)
        output = model.conv_layers(dummy_input)
        flattened_output = output.view(output.size(0), -1)
        print(f"Test {i+1}:")
        print(f"Flattened Size: {flattened_output.size(1)}")
        print(f"Half Flattened Size: {int(flattened_output.size(1) / 2)}")